## UMI example

This is an example taken from the pRESTO documentation, linked [here](http://presto.readthedocs.io/en/latest/workflows/Stern2014_Workflow.html). The abstract of the paper is below.

In [1]:
!esearch -db pubmed -query 25100741 | efetch -format abstract


1. Sci Transl Med. 2014 Aug 6;6(248):248ra107. doi: 10.1126/scitranslmed.3008879.

B cells populating the multiple sclerosis brain mature in the draining cervical
lymph nodes.

Stern JN(1), Yaari G(2), Vander Heiden JA(3), Church G(4), Donahue WF(5), Hintzen
RQ(6), Huttner AJ(7), Laman JD(8), Nagra RM(9), Nylander A(1), Pitt D(1), Ramanan
S(1), Siddiqui BA(1), Vigneault F(10), Kleinstein SH(11), Hafler DA(12), O'Connor
KC(13).

Author information: 
(1)Department of Neurology, Yale School of Medicine, New Haven, CT 06511, USA.
(2)Department of Pathology, Yale School of Medicine, New Haven, CT 06511, USA.
Bioengineering Program, Faculty of Engineering, Bar-Ilan University, Ramat Gan
52900, Israel. (3)Interdepartmental Program in Computational Biology and
Bioinformatics, Yale University, New Haven, CT 06511, USA. (4)Department of
Genetics, Harvard Medical School, Boston, MA 02115, USA. (5)AbVitro Incorporated,
Boston, MA 02210, USA. (6)Department of Neurology, Erasmus MC, University Medi

The read layout is shown below.

![](http://presto.readthedocs.io/en/latest/_images/Stern2014_ReadConfiguration.svg)

The workflow for processing the data is shown below.

![](http://presto.readthedocs.io/en/latest/_images/Stern2014_Flowchart.svg)

## Obtaining the data

Use `fastq-dump` to obtain the sequence data from accession SRR1383456, by running the cell below.

In [2]:
!fastq-dump --split-files --readids SRR1383456

Read 63073 spots for SRR1383456
Written 63073 spots for SRR1383456


## Filtering by quality

Use `FilterSeq.py quality` with a minimum (mean) quality score of 20 to filter the paired end reads. The usage information is given below.

In [3]:
!FilterSeq.py quality -h

usage: FilterSeq.py quality [-h] -s SEQ_FILES [SEQ_FILES ...] [--fasta]
                            [--failed] [--log LOG_FILE] [--nproc NPROC]
                            [--outdir OUT_DIR] [--outname OUT_NAME]
                            [-q MIN_QUAL] [--inner]

optional arguments:
  -h, --help            show this help message and exit
  -s SEQ_FILES [SEQ_FILES ...]
                        A list of FASTA/FASTQ files containing sequences to
                        process. (default: None)
  --fasta               Specify to force output as FASTA rather than FASTQ.
                        (default: None)
  --failed              If specified create files containing records that fail
                        processing. (default: False)
  --log LOG_FILE        Specify to write verbose logging to a file. May not be
                        specified with multiple input files. (default: None)
  --nproc NPROC         The number of simultaneous computational processes to
                     

Run the cell below to filter read 1.

In [4]:
!FilterSeq.py quality -s SRR1383456_1.fastq -q 20 --outname SRR1383456_R1 --log SRR1383456.quality.R1.log

   START> FilterSeq
 COMMAND> quality
    FILE> SRR1383456_1.fastq
   INNER> False
MIN_QUAL> 20.0
   NPROC> 4

PROGRESS> 10:34:54 [####################] 100% (63,073) 0.4 min

   OUTPUT> SRR1383456_R1_quality-pass.fastq
SEQUENCES> 63073
     PASS> 57958
     FAIL> 5115
      END> FilterSeq



**Now repeat the above, but for the read 2.**

In [5]:
!FilterSeq.py quality -s SRR1383456_2.fastq -q 20 --outname SRR1383456_R2 --log SRR1383456.quality.R2.log

   START> FilterSeq
 COMMAND> quality
    FILE> SRR1383456_2.fastq
   INNER> False
MIN_QUAL> 20.0
   NPROC> 4

PROGRESS> 10:35:13 [####################] 100% (63,073) 0.3 min

   OUTPUT> SRR1383456_R2_quality-pass.fastq
SEQUENCES> 63073
     PASS> 50234
     FAIL> 12839
      END> FilterSeq



## Masking primers

Now cut the primers (`Stern2014_CPrimers.fasta`) from read 1 We know where the primers start (15 for read 1), so we can use `MaskPrimers.py score` rather than `MaskPrimers.py align`. Read 1 is bar-coded **so you need to specify the `--barcode` option** to extract the barcode region.

In [6]:
!MaskPrimers.py score -h

usage: MaskPrimers.py score [-h] -s SEQ_FILES [SEQ_FILES ...] [--fasta]
                            [--failed] [--log LOG_FILE]
                            [--delim DELIMITER DELIMITER DELIMITER]
                            [--nproc NPROC] [--outdir OUT_DIR]
                            [--outname OUT_NAME] -p PRIMER_FILE
                            [--mode {cut,mask,trim,tag}]
                            [--maxerror MAX_ERROR] [--revpr] [--barcode]
                            [--start START]

optional arguments:
  -h, --help            show this help message and exit
  -s SEQ_FILES [SEQ_FILES ...]
                        A list of FASTA/FASTQ files containing sequences to
                        process. (default: None)
  --fasta               Specify to force output as FASTA rather than FASTQ.
                        (default: None)
  --failed              If specified create files containing records that fail
                        processing. (default: False)
  --log LOG_FILE      

Run the cell below to mask the reverse primers and extract the barcode.

In [7]:
!MaskPrimers.py score -s SRR1383456_R1_quality-pass.fastq -p Stern2014_CPrimers.fasta \
    --start 15 --mode cut --barcode --outname SRR1383456_R1 --log SRR1383456.REV.log

      START> MaskPrimers
    COMMAND> score
   SEQ_FILE> SRR1383456_R1_quality-pass.fastq
PRIMER_FILE> Stern2014_CPrimers.fasta
       MODE> cut
    BARCODE> True
  MAX_ERROR> 0.2
  START_POS> 15
 REV_PRIMER> False
      NPROC> 4

PROGRESS> 10:35:41 [####################] 100% (57,958) 0.4 min

   OUTPUT> SRR1383456_R1_primers-pass.fastq
SEQUENCES> 57958
     PASS> 51952
     FAIL> 6006
      END> MaskPrimers



**Using the above as a guide, complete the below cell to mask the forward primer from read 2 (`Stern2014_VPrimers.fasta`) starting from position 0.**

In [8]:
!MaskPrimers.py score -s SRR1383456_R2_quality-pass.fastq -p Stern2014_VPrimers.fasta \
    --start 0 --mode mask --outname SRR1383456_R2 --log SRR1383456.FWD.log

      START> MaskPrimers
    COMMAND> score
   SEQ_FILE> SRR1383456_R2_quality-pass.fastq
PRIMER_FILE> Stern2014_VPrimers.fasta
       MODE> mask
    BARCODE> False
  MAX_ERROR> 0.2
  START_POS> 0
 REV_PRIMER> False
      NPROC> 4

PROGRESS> 10:36:37 [####################] 100% (50,234) 0.9 min

   OUTPUT> SRR1383456_R2_primers-pass.fastq
SEQUENCES> 50234
     PASS> 48076
     FAIL> 2158
      END> MaskPrimers



## Copying over the barcodes

The barcode region is only annotated in read 1, so we need to copy this over to read 2, which we do with `PairSeq.py`.

In [9]:
!PairSeq.py -h

usage: PairSeq.py [-h] -1 SEQ_FILES_1 [SEQ_FILES_1 ...] -2 SEQ_FILES_2
                  [SEQ_FILES_2 ...] [--fasta] [--failed]
                  [--delim DELIMITER DELIMITER DELIMITER] [--outdir OUT_DIR]
                  [--outname OUT_NAME] [--version]
                  [--1f FIELDS_1 [FIELDS_1 ...]]
                  [--2f FIELDS_2 [FIELDS_2 ...]]
                  [--coord {illumina,solexa,sra,454,presto}]

Sorts and matches sequence records with matching coordinates across files

optional arguments:
  -h, --help            show this help message and exit
  -1 SEQ_FILES_1 [SEQ_FILES_1 ...]
                        An ordered list of FASTA/FASTQ files containing
                        head/primary sequences. (default: None)
  -2 SEQ_FILES_2 [SEQ_FILES_2 ...]
                        An ordered list of FASTA/FASTQ files containing
                        tail/secondary sequences. (default: None)
  --fasta               Specify to force output as FASTA rather than FASTQ.
             

Run the below cell to copy over the annotation.

In [10]:
!PairSeq.py -1 SRR1383456_R1_primers-pass.fastq -2 SRR1383456_R2_primers-pass.fastq \
    --1f BARCODE --coord sra

     START> PairSeq
     FILE1> SRR1383456_R1_primers-pass.fastq
     FILE2> SRR1383456_R2_primers-pass.fastq
  FIELDS_1> BARCODE
  FIELDS_2> None
COORD_TYPE> sra

PROGRESS> 10:36:40 [Done                ] 0.0 min

PROGRESS> 10:36:52 [####################] 100% (48,076) 0.2 min

   OUTPUT1> SRR1383456_R1_primers-pass_pair-pass.fastq
   OUTPUT2> SRR1383456_R2_primers-pass_pair-pass.fastq
SEQUENCES1> 51952
SEQUENCES2> 48076
      PASS> 43457
       END> PairSeq



## Building a consensus based  on the barcodes

We now build a consensus sequence for the sequences with a particular barcode. We do this separately for the paired sequences, `SRR1383456_R1_primers-pass_pair-pass.fastq` and `SRR1383456_R2_primers-pass_pair-pass.fastq` using `BuildConsensus.py`.

In [11]:
!BuildConsensus.py -h

usage: BuildConsensus.py [-h] -s SEQ_FILES [SEQ_FILES ...] [--fasta]
                         [--failed] [--log LOG_FILE]
                         [--delim DELIMITER DELIMITER DELIMITER]
                         [--nproc NPROC] [--outdir OUT_DIR]
                         [--outname OUT_NAME] [--version] [-n MIN_COUNT]
                         [--bf BARCODE_FIELD] [-q MIN_QUAL] [--freq MIN_FREQ]
                         [--maxgap MAX_GAP] [--pf PRIMER_FIELD]
                         [--prcons PRIMER_FREQ]
                         [--cf COPY_FIELDS [COPY_FIELDS ...]]
                         [--act {min,max,sum,set,majority} [{min,max,sum,set,majority} ...]]
                         [--dep]
                         [--maxdiv MAX_DIVERSITY | --maxerror MAX_ERROR]

Builds a consensus sequence for each set of input sequences

optional arguments:
  -h, --help            show this help message and exit
  -s SEQ_FILES [SEQ_FILES ...]
                        A list of FASTA/FASTQ files containi

Run the cell below to build a consensus from read 1.

In [12]:
!BuildConsensus.py -s SRR1383456_R1_primers-pass_pair-pass.fastq --bf BARCODE --pf PRIMER \
    --prcons 0.6 --maxerror 0.1 --maxgap 0.5 --outname SRR1383456_R1 --log SRR1383456.consensus.R1.log

           START> BuildConsensus
            FILE> SRR1383456_R1_primers-pass_pair-pass.fastq
   BARCODE_FIELD> BARCODE
       MIN_COUNT> 1
   MIN_FREQUENCY> 0.6
     MIN_QUALITY> 0
         MAX_GAP> 0.5
    PRIMER_FIELD> PRIMER
PRIMER_FREQUENCY> 0.6
       MAX_ERROR> 0.1
   MAX_DIVERSITY> None
       DEPENDENT> False
     COPY_FIELDS> None
    COPY_ACTIONS> None
           NPROC> 4

PROGRESS> 10:37:27 [####################] 100% (6,625) 0.5 min

   OUTPUT> SRR1383456_R1_consensus-pass.fastq
SEQUENCES> 43457
     SETS> 6625
     PASS> 6575
     FAIL> 50
      END> BuildConsensus



**Now complete the below cell to build a consensus for read 2.**

In [13]:
!BuildConsensus.py -s SRR1383456_R2_primers-pass_pair-pass.fastq --bf BARCODE --pf PRIMER \
    --maxerror 0.1 --maxgap 0.5 --outname SRR1383456_R2 --log SRR1383456.consensus.R2.log

           START> BuildConsensus
            FILE> SRR1383456_R2_primers-pass_pair-pass.fastq
   BARCODE_FIELD> BARCODE
       MIN_COUNT> 1
   MIN_FREQUENCY> 0.6
     MIN_QUALITY> 0
         MAX_GAP> 0.5
    PRIMER_FIELD> PRIMER
PRIMER_FREQUENCY> None
       MAX_ERROR> 0.1
   MAX_DIVERSITY> None
       DEPENDENT> False
     COPY_FIELDS> None
    COPY_ACTIONS> None
           NPROC> 4

PROGRESS> 10:38:05 [####################] 100% (6,625) 0.6 min

   OUTPUT> SRR1383456_R2_consensus-pass.fastq
SEQUENCES> 43457
     SETS> 6625
     PASS> 6526
     FAIL> 99
      END> BuildConsensus



Now run the following cell to pair the sequences that passed the consensus building.

In [14]:
!PairSeq.py -1 SRR1383456_R1_consensus-pass.fastq -2 SRR1383456_R2_consensus-pass.fastq \
    --coord presto

     START> PairSeq
     FILE1> SRR1383456_R1_consensus-pass.fastq
     FILE2> SRR1383456_R2_consensus-pass.fastq
  FIELDS_1> None
  FIELDS_2> None
COORD_TYPE> presto

PROGRESS> 10:38:05 [Done                ] 0.0 min

PROGRESS> 10:38:07 [####################] 100% (6,526) 0.0 min

   OUTPUT1> SRR1383456_R1_consensus-pass_pair-pass.fastq
   OUTPUT2> SRR1383456_R2_consensus-pass_pair-pass.fastq
SEQUENCES1> 6575
SEQUENCES2> 6526
      PASS> 6514
       END> PairSeq



## Assembling mate pairs

In [15]:
!AssemblePairs.py align -h

usage: AssemblePairs.py align [-h] -1 SEQ_FILES_1 [SEQ_FILES_1 ...] -2
                              SEQ_FILES_2 [SEQ_FILES_2 ...] [--fasta]
                              [--failed] [--log LOG_FILE]
                              [--delim DELIMITER DELIMITER DELIMITER]
                              [--nproc NPROC] [--outdir OUT_DIR]
                              [--outname OUT_NAME]
                              [--coord {illumina,solexa,sra,454,presto}]
                              [--rc {head,tail,both}]
                              [--1f HEAD_FIELDS [HEAD_FIELDS ...]]
                              [--2f TAIL_FIELDS [TAIL_FIELDS ...]]
                              [--alpha ALPHA] [--maxerror MAX_ERROR]
                              [--minlen MIN_LEN] [--maxlen MAX_LEN]
                              [--scanrev]

optional arguments:
  -h, --help            show this help message and exit
  -1 SEQ_FILES_1 [SEQ_FILES_1 ...]
                        An ordered list of FASTA/FASTQ files co

In [16]:
!AssemblePairs.py align -1 SRR1383456_R2_consensus-pass_pair-pass.fastq \
    -2 SRR1383456_R1_consensus-pass_pair-pass.fastq --coord presto --rc tail \
    --1f CONSCOUNT --2f CONSCOUNT PRCONS --outname SRR1383456 --log SRR1383456.assemble.log

/usr/local/bin/AssemblePairs.py:112: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  p_matrix[x, i:] = 1 - stats.binom.cdf(x - 1, k[i:], 0.25) - stats.binom.pmf(x, k[i:], 0.25) / 2.0
/usr/local/bin/AssemblePairs.py:131: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  z_matrix[x, j:] = (x - k[j:]/4.0)/np.sqrt(3.0/16.0*k[j:])
       START> AssemblePairs
     COMMAND> align
       FILE1> SRR1383456_R2_consensus-pass_pair-pass.fastq
       FILE2> SRR1383456_R1_consensus-pass_pair-pass.fastq
  COORD_TYPE> presto
       ALPHA> 1e-05
   MAX_ERROR> 0.3
     MIN_LEN> 8
     MAX_LEN> 1000
SCAN_REVERSE> False
       NPROC> 4

PROGRESS> 10:38:52 [####################] 100% (6,514) 0.7 min

OUTPUT> SRR1383456_assemble-pass.fastq
 PAIRS> 6514
  PASS> 2773
  FAIL> 3741
   END> AssemblePairs



## Deduplication and filtering

Let's take a look at the resulting sequences.

In [17]:
!head SRR1383456_assemble-pass.fastq

@CTCCCGGAGGATTAA|CONSCOUNT=1,1|PRCONS=20080924-IGHA
NNNNNNNNNNNNNNNNNNNNNNGCTCAGCCCACTGTGTCTGCCCGATGCTCACCTGTCCAGAGGCCAACGCTACCAAGGTGAGGGGTGTGGGATGTGAAGGGGAGTGGGGAGGAGGCCTCGCCTTGAACATCCTTGTTCTCTGCCCCAGGCCTGTGGGTCAGATGGAGTCACATACGTCAACGAGTGTCAGCTGAAGACCATCGCCTGCCGCCAGGGCCTGCAAATCTCTATCCAGAGCCTGGGCCCGTGCCAGGGTGAGGCCTGACGGCCACTGCCCCAGAACTGACCAGGAAGGCCTGACGCTGCCCTAAATCCAGCCCCACCCGCCCTGAGCCACCTGACCCTGTCCCAACCGGGCC
+
!!!!!!!!!!!!!!!!!!!!!!HHFHFHFHHFHHB0CGFHDDGHHH=ED/EFHFF@ACFCEGFDEFHHHHHDEEHHHHF5C?C;CC5<4=CEDCC,@FFCBDB3@3@@DE@B<BABBCCEE??EEEEEEEEEEEECC:?CE::EEEE8?A(0*.8?*8,:8?:?EE::*0AC???AA(8)8AAAEEE#####################GACGGGGEGEC:C:::E=:?GEEEECC9?EGEC4<DGGGGGEECGGGGEC?GEGGED<AEEC?EGGGGGEEGEGECECEGEEGGGGGECGEGE4C?8CEEEGGEE?GEDC>DD;0DGGGGGEEEEEECEGGGEEGEGEEC(GGDDDGB9)GH
@GGAGGTGTGGACGGG|CONSCOUNT=8,8|PRCONS=20080924-IGHG
NNNNNNNNNNNNNNNNNNNNTGGCCACTATGCCTCGCAGCAGATCAGGCAGATCGCGAGTCAGCTGGAGCAGGAGTGGAAGGCGTTTGCGGCAGCCCTGGATGAGCGGAGCACCTTGCTGGACATGTCCTCCATTTTCCACCAGAAGGCCGAAAAGTATATGAGCAACGTGG

As you can see, there are two entries for the number of sequences used for each consensus. The following command replaces this pair with the minimum of the two.

In [18]:
!ParseHeaders.py collapse -s SRR1383456_assemble-pass.fastq -f CONSCOUNT --act min

  START> ParseHeaders
COMMAND> collapse
   FILE> SRR1383456_assemble-pass.fastq
ACTIONS> min
 FIELDS> CONSCOUNT

PROGRESS> 10:38:53 [####################] 100% (2,773) 0.0 min

   OUTPUT> SRR1383456_assemble-pass_reheader.fastq
SEQUENCES> 2773
      END> ParseHeaders



The following command removes duplicate sequences.

In [19]:
!CollapseSeq.py -s SRR1383456_assemble-pass_reheader.fastq -n 20 --inner --uf PRCONS \
    --cf CONSCOUNT --act sum --outname SRR1383456

       START> CollapseSeq
        FILE> SRR1383456_assemble-pass_reheader.fastq
 MAX_MISSING> 20
 UNIQ_FIELDS> PRCONS
 COPY_FIELDS> CONSCOUNT
COPY_ACTIONS> sum
   MAX_FIELD> None
   MIN_FIELD> None
       INNER> True
KEEP_MISSING> False

MISSING>  0
PROGRESS> 10:38:53 [####################] 100% (2,773) 0.0 min

MISSING>  1
PROGRESS> 10:38:53 [####################] 100% (443) 0.0 min

MISSING>  2
PROGRESS> 10:38:53 [####################] 100% (285) 0.0 min

MISSING>  3
PROGRESS> 10:38:54 [####################] 100% (219) 0.0 min

MISSING>  4
PROGRESS> 10:38:54 [####################] 100% (176) 0.0 min

MISSING>  5
PROGRESS> 10:38:54 [####################] 100% (139) 0.0 min

MISSING>  6
PROGRESS> 10:38:54 [####################] 100% (120) 0.0 min

MISSING>  7
PROGRESS> 10:38:54 [####################] 100% (100) 0.0 min

MISSING>  8
PROGRESS> 10:38:54 [####################] 100% (87) 0.0 min

MISSING>  9
PROGRESS> 10:38:54 [####################] 100% (77) 0.0 min

MISSING>  10
PROGRESS>

We will also convert the sequences to FASTA format.

In [20]:
from Bio import SeqIO
SeqIO.convert("SRR1383456_collapse-unique.fastq","fastq","SRR1383456_collapse-unique.fasta","fasta")

1585

In [21]:
!head SRR1383456_collapse-unique.fasta

>TTTCAGGAGGCGTTA|PRCONS=20080924-IGHA|CONSCOUNT=1|DUPCOUNT=1
NNNNNNNNNNNNNNNNAGTCTGGGCAGAGCCACTGCTCAAGCCAATGTGGATCTCTGAGC
ACCCCAGCACCAGCTGCCACAGCCGGCCAACAAGGTAGGCCAGGCTCTGTCATGTGCCCC
TAGGATAGGGGCCATATCCATGATGCTGGGGAGCAGACAGACTGCAGGCCTCGCCTCCAC
TGCACCTTGCCAGGCAAGGCCCACTGGCCTGGGCCTCCATCGCAGCCATCCACCCCCGAC
CAGCCCCCAG
>CCTATACGGCACGGG|PRCONS=20080924-IGHA|CONSCOUNT=2|DUPCOUNT=1
NNNNNNNNNNNNNNNNNNNNAACTGCAAAACTGCAAGTTGCACACAGGCNGAGATGACGG
AGCAACCCCTGACCCTCCGTGCCACTCACCCTACCCGCACACACACTTGCCACGTGGACC
CTGAAGCCAGTGCCTGGGCGCCCNAGTCAGGCAGTCCACACAGCAGTGGCACCAGGGTGA
